In [1]:
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout

from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils

from sklearn.cross_validation import cross_val_score, KFold
from sklearn.cross_validation import train_test_split

from sklearn.grid_search import GridSearchCV

from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
dataframe = pd.read_csv('/Users/johannes/Documents/Unique/Datasets/Iris/Iris.csv')

In [3]:
dataframe.head(3)

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa


In [4]:
data = dataframe.values

In [5]:
type(data)

numpy.ndarray

In [6]:
X = data[:, 0:4]
y = data[:, 4]

In [7]:
# encode class values as integers
encoder = LabelEncoder()
encoded_Y = encoder.fit_transform(y)

In [8]:
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [9]:
# define baseline model
def mlp_model(neurons=4, optimizer='adam'):
    # create model
    model = Sequential()
    
    model.add(Dense(neurons, input_dim=4, init='uniform'))
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))
    
    model.add(Dense(neurons, init='uniform'))
    model.add(Activation('tanh'))
    model.add(Dropout(0.5))
    
    model.add(Dense(3, init='uniform'))
    model.add(Activation('softmax'))
    
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    
    return model

In [10]:
# For GridSearchCV
keras_model = KerasClassifier(build_fn = mlp_model, verbose=0)

In [11]:
# define the grid search parameters
# total_epochs = [100, 200]
# total_batch = [5, 15, 20]
# optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
# neurons = [4, 8, 16, 32, 64]

total_epochs = [200]
total_batch = [5]
optimizer = ['Adam']
neurons = [64]

param_grid = dict(nb_epoch=total_epochs, batch_size=total_batch, neurons=neurons, optimizer=optimizer)

grid = GridSearchCV(estimator=keras_model, param_grid=param_grid, cv=10, n_jobs=-1)
grid_result = grid.fit(X, dummy_y)

In [12]:
# summarize results
print("Best score: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best score: 0.973333 using {'nb_epoch': 200, 'neurons': 64, 'optimizer': 'Adam', 'batch_size': 5}


In [13]:
# For Cross Validation
# keras_model = KerasClassifier(build_fn = mlp_model, nb_epoch=200, batch_size=5, verbose=0)

# X_train, X_test, y_train, y_test = train_test_split(X, dummy_y, test_size=0.3)

# estimator.fit(X_train, y_train)

# predictions = estimator.predict(X_test)

# results = cross_val_score(keras_model, X, dummy_y, cv=10)

# print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))